In [ ]:
import os
import sys
import nest_asyncio
nest_asyncio.apply()
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
from trusteval.dimension.ethics.ethics_llm import pipeline

base_dir = os.getcwd() + "./test/ethics/"
os.makedirs(base_dir, exist_ok=True)

In [ ]:
from trusteval import download_metadata
download_metadata(
    section='ethics_llm',
    output_path=base_dir
)

In [ ]:
await pipeline.pipeline(base_dir=base_dir)

In [ ]:
from trusteval import contextual_variator_cli
import shutil

source_config = "../section/ethics/ethics_llm/file_config_ethics.json"
target_config = os.path.join(base_dir,'data',"file_config.json")

if os.path.exists(source_config):
    shutil.copy2(source_config, target_config)
    print(f"Successfully copied file_config to {target_config}")
else:
    print("Warning: Source file_config not found")

dataset_folder = os.path.join(base_dir,'data')
contextual_variator_cli(
    dataset_folder=dataset_folder
)

In [ ]:
from trusteval.src.response_generator.lm_response import generate_responses
import shutil

source_config = "../trusteval/dimension/ethics/ethics_llm/data/file_config.json"
target_config = os.path.join(base_dir,'data',"file_config.json")

if os.path.exists(source_config):
    shutil.copy2(source_config, target_config)
    print(f"Successfully copied file_config to {target_config}")
else:
    print("Warning: Source file_config not found")
    
dataset_folder = os.path.join(base_dir,'data')

async_list = ['o1-preview','o1-mini','qwq-32B']

await generate_responses(
        data_folder=dataset_folder,
        request_type="llm",
        async_list=async_list,
        prompt_key="enhanced_prompt",
        result_key="responses"
    )

print("All files processed.")

In [ ]:
from trusteval.src.evaluation import judge_responses

config_path = os.path.join(parent_dir, 'trusteval/src/config/judge_prompt.yaml')
async_judge_model = ['gpt-4o-mini']
response_key = ['responses']
judge_key = 'judge'
judge_type='llm'
await judge_responses(
        data_folder=dataset_folder,
        async_judge_model=async_judge_model,
        target_models=async_list,
        judge_type=judge_type
    )

In [ ]:
from trusteval.src.evaluation import lm_metric

lm_metric(
    base_dir=base_dir,
    aspect="ethics_llm",
    model_list=async_list
)